In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [3]:
x_test=np.load('../Data/x_test.npy')
y_test=np.load('../Data/y_test.npy')
print(x_test.shape, y_test.shape)

(3600, 2) (3600,)


In [4]:
model=load_model('../NN_model/13-0.9664.h5', compile=False)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 32)                96        
                                                                 
 dropout_6 (Dropout)         (None, 32)                0         
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dropout_7 (Dropout)         (None, 16)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 641 (2.50 KB)
Trainable params: 641 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if y_pred[i]>=0.5:
            count1+=1
print(count,count1)

113/113 [==============================] - 0s 620us/step
1767 1709


## HLS4ML

In [6]:
import os
import hls4ml
import plotting
from hls4ml.model import profiling

In [7]:
# os.environ['PATH']+= os.pathsep + '/opt/Xilinx/Vivado/2022.1/bin'
os.environ['PATH']+= os.pathsep + '/opt/Xilinx/Vivado/2018.3/bin'

In [8]:
config = hls4ml.utils.config_from_keras_model(model, granularity='name')
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")

Interpreting Sequential
Topology:
Layer name: input_4, layer type: InputLayer, input shapes: [[None, 2]], output shape: [None, 2]
Layer name: dense_9, layer type: Dense, input shapes: [[None, 2]], output shape: [None, 32]
Layer name: dense_10, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 16]
Layer name: dense_11, layer type: Dense, input shapes: [[None, 16]], output shape: [None, 1]
-----------------------------------
Model
  Precision:         fixed<16,6>
  ReuseFactor:       1
  Strategy:          Latency
  BramFactor:        1000000000
  TraceOutput:       False
LayerName
  input_4
    Trace:           False
    Precision
      result:        fixed<16,6>
  dense_9
    Trace:           False
    Precision
      result:        fixed<16,6>
      weight:        fixed<16,6>
      bias:          fixed<16,6>
  dense_9_relu
    Trace:           False
    Precision
      result:        fixed<16,6>
  dense_10
    Trace:           False
    Precision
      result:       

In [9]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='../model_2/myproject_prj',part='xc7vx485tffg1761-2',
)

Interpreting Sequential
Topology:
Layer name: input_4, layer type: InputLayer, input shapes: [[None, 2]], output shape: [None, 2]
Layer name: dense_9, layer type: Dense, input shapes: [[None, 2]], output shape: [None, 32]
Layer name: dense_10, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 16]
Layer name: dense_11, layer type: Dense, input shapes: [[None, 16]], output shape: [None, 1]
Creating HLS model


In [12]:
hls_model.compile()
y_hls=hls_model.predict(np.ascontiguousarray(x_test))

Writing HLS project
Done


In [13]:
count=0
count1=0
for i in range(y_hls.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_hls[i]<=0.5:
            count1+=1
print(count,count1)

1833 1770


In [14]:
hls_model.build()


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2018.3 (64-bit)
  **** SW Build 2405991 on Thu Dec  6 23:36:41 MST 2018
  **** IP Build 2404404 on Fri Dec  7 01:43:56 MST 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2018.3/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2018.3/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'neelvora' on host 'new-begcbp-desktop' (Linux_x86_64 version 5.15.0-78-generic) on Wed Oct 18 13:27:59 PDT 2023
INFO: [HLS 200-10] On os Ubuntu 22.04.3 LTS
INFO: [HLS 200-10] In directory '/home/neelvora/Projects/Quantum/model_2/myproject_prj'
INFO: [HLS 200-10] Creating and opening project '/home/neelvora/Projects/Quantum/model_2/myproject_prj/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the project
INFO: [HLS 200-10] Adding test bench file 'myproject_test.cpp' to the project
INFO: [HLS 200-10] Adding test ben

{'CSimResults': [['0.900391']],
 'CSynthesisReport': {'TargetClockPeriod': '5.00',
  'EstimatedClockPeriod': '4.122',
  'BestLatency': '15',
  'WorstLatency': '15',
  'IntervalMin': '1',
  'IntervalMax': '1',
  'BRAM_18K': '1',
  'DSP': '482',
  'FF': '14168',
  'LUT': '18342',
  'AvailableBRAM_18K': '2060',
  'AvailableDSP': '2800',
  'AvailableFF': '607200',
  'AvailableLUT': '303600'}}

In [16]:
hls4ml.report.read_vivado_report('../model_2/myproject_prj')

Found 1 solution(s) in ../model_2/myproject_prj/myproject_prj.
Reports for solution "solution1":

C SIMULATION RESULT:
INFO: [SIM 2] *************** CSIM start ***************
INFO: [SIM 4] CSIM will launch GCC as the compiler.
   Compiling ../../../../myproject_test.cpp in debug mode
   Compiling ../../../../firmware/myproject.cpp in debug mode
   Generating csim.exe
INFO: Unable to open input/predictions file, using default input.
0.900391 
INFO: Saved inference results to file: tb_data/csim_results.log
INFO: [SIM 1] CSim done with 0 errors.
INFO: [SIM 3] *************** CSIM finish ***************

SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Wed Oct 18 13:28:48 2023

* Version:        2018.3 (Build 2405991 on Thu Dec 06 23:56:15 MST 2018)
* Project:        myproject_prj
* Solution:       solution1
* Product family: virtex7
* Target device:  xc7vx485tffg1761-2


== Performance Estimates
+ Timing (ns): 
    * Summary: 
    +--------+-------+----------+----